In [4]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *
    
# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  # Europe
site = 'miadhu_mv'
c = Crawl()  # creating object

# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)

# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_article_url = 0
publish_source = 'miadhu.mv'
country = 'Maldives'
language = 'English'
images_path = []

rss_pages = c.download_page('https://miadhu.mv/en')
for i in rss_pages.split('<div class="row">')[2:-2]:
    source_link = c.scrap('<a\s*href="(.*?)"', i)  #
    print(source_link)
    source_link_query = {'source_link':source_link}
    dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
    
    if dic:
        duplicate_data += 1
        continue    
        
    logger.info(f'Fetching {source_link}\n')
    time.sleep(random.randint(1,3))

    page = c.download_page(source_link)

    if page.startswith('Unable to fetch'):
        logger.info(page) # writes error message with error code
        unable_to_fetch_article_url += 1
        continue

    source_headline = c.scrap('<h1.*?>(.*?)</h1>', page)
    source_headline = re.sub('&.*?;','',source_headline)
    print(source_headline)
    if not source_headline:
        logger.info(f'Skipping due to headline {source_link}\n')
        skipped_due_to_headline += 1
        continue
   
    pub_date, publish_time = '', ''            
            
    try: 
        date_time_str = c.scrap('class="article-date">(.*?)</', page)
        date_time_str = re.sub('[^\w+]','',date_time_str)
        date_time_obj = datetime.strptime(date_time_str, '%Y%m%d%H%M%S')
        ist_date_time = date_time_obj + timedelta(hours = 0,minutes = 30)  
        ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')  
        pub_date = ist_date_time[:10]
        publish_time = ist_date_time[11:]
        print(pub_date)

    except:
        pass
  
    if not pub_date:
        logger.info(f'Skipping due to date {source_link}\n')
        skipped_due_to_date += 1
        continue

#     if pub_date != created_on:
#         break 
        
    source_content = c.scrap('<div\s*class="read-dv-text">(.*?)</p></div>',page)
    source_content = re.sub('&.*?;','',source_content)
    source_content = c.strip_html(source_content)
    print(source_content)
    continue

   
    if not source_content:
        logger.info(f'Skipping due to content {source_link}\n')
        skipped_due_to_content += 1
        continue
    
    journalist =  ''
    if not journalist: journalist = 'NA'
        
        
  # current date and time 
    harvest_time = datetime.now().strftime("%H:%M:%S")

    # temp link
    temp_link = source_link

    # headline and content 
    headline = source_headline
    content = source_content

    # overall_tonality
    overall_tonality = ''

    # word count
    word_count = len((source_headline + ' ' + source_content).split())

    html_content = ''

    # image_urls
    image_urls = []

   # storing the above data in a dictionary
    clientdata ={
                    "client_master" : client_id, 
                    "articleid":client_id,
                    "medium":'Web' ,
                    "searchkeyword":[],
                    "entityname" : [] ,
                    "process_flage":"1",
                    "na_flage":"0",
                    "na_reason":"",
                    "qc_by":"",
                    "qc_on":"",
                    "location":"",
                    "spokeperson":"",
                    "quota":"",
                    "overall_topics":"",
                    "person":"",
                    "overall_entites":"",
                    "overall_tonality": overall_tonality,
                    "overall_wordcount":word_count,
                    "article_subjectivity":"",
                    "article_summary":"",
                    "pub_date":pub_date,
                    "publish_time":publish_time,
                    "harvest_time":harvest_time,
                    "temp_link":temp_link,
                    "publish_source": publish_source,
                    "programme":'null',
                    "feed_class":"News",
                    "publishing_platform":"",
                    "klout_score":"",
                    "journalist":journalist,
                    "headline":headline,
                    "content":content,
                    "source_headline":source_headline,
                    "source_content":source_content,
                    "language":language,
                    "presence":'null',
                    "clip_type":'null',
                    "prog_slot":'null',
                    "op_ed":'0',
                    "location_mention":'',
                    "source_link":source_link,
                    "author_contact":'',
                    "author_emailid":'',
                    "author_url":'',
                    "city":'',
                    "state":'',
                    "country":country,
                    "source":publish_source,
                    "foot_fall":'',
                    "created_on":created_on,
                    "active":'1',
                    'crawl_flag':2,
                    "images_path":images_path,
                    "html_content":html_content
                } 

#     cl_data.insert_one(clientdata)  
    no_of_data += 1


logger.info('Iteration complete\n')   
logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch article url: {unable_to_fetch_article_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n')

https://miadhu.mv/article/en/13250
President sends felicitations on Commonwealth Day
2021-03-08
President Ibrahim Mohamed Solih has sent a message of felicitations commemorating this year's Commonwealth Day.
The Presidents message reads as follows:
Today on Commonwealth Day, we celebrate our shared values and strong bonds, uniting people over 54 countries and across 6 continents.
This years theme, Delivering a Common Future: Connecting, Innovating, Transforming, reflects the strong ties that bind our commonwealth family of 2.4 billion people together. These ties have been strongly evident throughout the ongoing Covid-19 pandemic, in our support and goodwill towards one another in these extraordinary and difficult circumstances.
It has now been one year since the Maldives re-joined the Commonwealth, and we are proud of all that we have accomplished together within this short time period. The Commonwealth has always been a champion of its member states, from the most populous to small is

Dhiraagu frontline staff vaccinated against COVID-19
2021-02-07
Dhiraagu frontline staff have received their first dose of the COVID-19 vaccine.
With the recent surge in COVID-19 cases in the country, health and safety of our customers and employees remain our top priority. Therefore, under the Governments initiative to vaccinate frontline staff, today we have received registered slots for Dhiraagu staff.
In the midst of the third wave of the COVID-19 pandemic, our frontline engineers and technicians visiting customer premises are continuing to play a critical role in maintaining our network infrastructure and ensuring services remain available to our customers. Also, our customer service centre staff continues to serve frontline regardless of the severity of the pandemic we are in today. We have been taking several measures in accordance to the advice from the Health Protection Agency (HPA) to minimize the risk of exposing our staff whilst working at sensitive areas.
Dhiraagu is commi

Maldives’ 100,000 Visitor since reopening borders enjoys Diving Holiday in Embudu Village.
2020-12-21
Maldives 100,000th tourist since the international border reopened in July, Ms. Jasmin Burki, is enjoying a diving holiday in Embudu Village, located in south Male Atoll.
Ms. Burki and her partner arrived in Male from Switzerland on 12 December, on an Emirates flight. Upon arrival, she was informed by MMPRC and Maldives Immigration officials that she was the 100,000th visitor since the international border reopened. She posed for photos and received prize gifts.
It was a bit unexpected but we are delighted and feel honoured to have been selected as the 100,000thguest, said Ms. Burki.
We got from the terminal to outside very quickly, it was a smooth process at the airport. We are very happy for being the lucky 100,000thguest, we were grateful for the gifts.
From Embudu, we got a free room upgrade to a Water Villa, a specialbeach dinner,and a specialcake. Thank you!
Ms. Burki said she ch